## 🛠 Step 2: Data Transformation

### Public Parks (Grünenanlage) Berlin

In [36]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from time import sleep
from sqlalchemy import create_engine
import re

In [3]:
# Load your CSV
df = pd.read_csv("/Users/dianaterraza/Desktop/webeet.io/layered-populate-data-pool-da/recreational_zones/sources/public_parks.csv", sep=';')

In [6]:
df.head()

,Technischer Schlüssel,Schlüssel,Objektnummer,Bezirk,Ortsteil,Art der Grünanlage,Name der Grünanlage,Namenszusatz der Grünanlage,Baujahr,letztes Sanierungsjahr,Größe in m² (Kataster),Widmung,Nummer des Planungsraumes,Name des Planungsraumes
0,00008100_001042bb,00008100:001042bb,00037,Reinickendorf,Frohnau,Grünanlage,"Im Fischgrund, ""Rosenanger""",Rosenanger,-,-,"16991,50",gewidmet,12400721,Frohnau Ost
1,00008100_00104621,00008100:00104621,01179,Reinickendorf,Lübars,Grünanlage,Klötzbecken bis Zabel-Krüger-Damm,einschl. Klötzbecken,-,-,"52224,60",gewidmet,12500929,Lübars
2,00008100_001044bd,00008100:001044bd,01074,Reinickendorf,Hermsdorf,Grünanlage,"Heidenheimer Str. (ab Friedrichsthaler Weg), W...",-,-,-,"3012,00",gewidmet,12400722,Hermsdorf West
3,00008100_00104620,00008100:00104620,01180,Reinickendorf,Lübars,Grünanlage,"Wittenauer Str., südl. AEG-Siedlung",-,-,-,"3374,20",gewidmet,12500929,Lübars
4,00008100_00104438,00008100:00104438,00476,Reinickendorf,Reinickendorf,Grünanlage,Kuhnpromenade u. Lindauer Allee 59/61,-,-,-,"3122,00",gewidmet,12100206,Humboldtstraße


In [7]:
df.columns

Index(['Technischer Schlüssel', 'Schlüssel', 'Objektnummer', 'Bezirk',
       'Ortsteil', 'Art der Grünanlage', 'Name der Grünanlage',
       'Namenszusatz der Grünanlage', 'Baujahr', 'letztes Sanierungsjahr',
       'Größe in m² (Kataster)', 'Widmung', 'Nummer des Planungsraumes',
       'Name des Planungsraumes'],
      dtype='object')

### Rename the Columns 

In [8]:
df.rename(columns={
    'Technischer Schlüssel': 'Technical ID',
    'Schlüssel': 'Key',
    'Objektnummer': 'Object Number',
    'Bezirk': 'neighborhood',
    'Ortsteil': 'Locality',
    'Art der Grünanlage': 'Type of Green Space',
    'Name der Grünanlage': 'Green Space Name',
    'Namenszusatz der Grünanlage': 'Name Extension',
    'Baujahr': 'Year Built',
    'letztes Sanierungsjahr': 'Last Renovation Year',
    'Größe in m² (Kataster)': 'Size sqm',
    'Widmung': 'Dedication',
    'Nummer des Planungsraumes': 'Planning Area Number',
    'Name des Planungsraumes': 'Planning Area Name'
}, inplace=True)


In [9]:
df.columns


Index(['Technical ID', 'Key', 'Object Number', 'neighborhood', 'Locality',
       'Type of Green Space', 'Green Space Name', 'Name Extension',
       'Year Built', 'Last Renovation Year', 'Size sqm', 'Dedication',
       'Planning Area Number', 'Planning Area Name'],
      dtype='object')

### Check for null values 

In [10]:
print(df.isnull().sum())

Technical ID            0
Key                     0
Object Number           0
neighborhood            0
Locality                0
Type of Green Space     0
Green Space Name        0
Name Extension          0
Year Built              0
Last Renovation Year    0
Size sqm                0
Dedication              0
Planning Area Number    0
Planning Area Name      0
dtype: int64


### Change the DataType of the columns for analysis 

In [11]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Technical ID          2556 non-null   object
 1   Key                   2556 non-null   object
 2   Object Number         2556 non-null   object
 3   neighborhood          2556 non-null   object
 4   Locality              2556 non-null   object
 5   Type of Green Space   2556 non-null   object
 6   Green Space Name      2556 non-null   object
 7   Name Extension        2556 non-null   object
 8   Year Built            2556 non-null   object
 9   Last Renovation Year  2556 non-null   object
 10  Size sqm              2556 non-null   object
 11  Dedication            2556 non-null   object
 12  Planning Area Number  2556 non-null   object
 13  Planning Area Name    2556 non-null   object
dtypes: object(14)
memory usage: 279.7+ KB


In [12]:
df.head()

,Technical ID,Key,Object Number,neighborhood,Locality,Type of Green Space,Green Space Name,Name Extension,Year Built,Last Renovation Year,Size sqm,Dedication,Planning Area Number,Planning Area Name
0,00008100_001042bb,00008100:001042bb,00037,Reinickendorf,Frohnau,Grünanlage,"Im Fischgrund, ""Rosenanger""",Rosenanger,-,-,"16991,50",gewidmet,12400721,Frohnau Ost
1,00008100_00104621,00008100:00104621,01179,Reinickendorf,Lübars,Grünanlage,Klötzbecken bis Zabel-Krüger-Damm,einschl. Klötzbecken,-,-,"52224,60",gewidmet,12500929,Lübars
2,00008100_001044bd,00008100:001044bd,01074,Reinickendorf,Hermsdorf,Grünanlage,"Heidenheimer Str. (ab Friedrichsthaler Weg), W...",-,-,-,"3012,00",gewidmet,12400722,Hermsdorf West
3,00008100_00104620,00008100:00104620,01180,Reinickendorf,Lübars,Grünanlage,"Wittenauer Str., südl. AEG-Siedlung",-,-,-,"3374,20",gewidmet,12500929,Lübars
4,00008100_00104438,00008100:00104438,00476,Reinickendorf,Reinickendorf,Grünanlage,Kuhnpromenade u. Lindauer Allee 59/61,-,-,-,"3122,00",gewidmet,12100206,Humboldtstraße


In [13]:
# Replace "-" with NaN before processing
df["Size sqm"] = df["Size sqm"].replace("-", np.nan)

# Convert "Size sqm" to float: remove thousand separators (dots), replace decimal commas with dots
df["Size sqm"] = df["Size sqm"] \
    .str.replace(".", "", regex=False) \
    .str.replace(",", ".", regex=False) \
    .astype(float)

# Convert year columns to numeric (invalid entries like "-" will become NaN)
df["Year Built"] = pd.to_numeric(df["Year Built"], errors="coerce")
df["Last Renovation Year"] = pd.to_numeric(df["Last Renovation Year"], errors="coerce")

# Convert planning area number to numeric (if needed)
df["Planning Area Number"] = pd.to_numeric(df["Planning Area Number"], errors="coerce")

# Convert selected text columns to category type to reduce memory usage
cat_columns = [
    "neighborhood", "Locality", "Type of Green Space",
    "Green Space Name", "Name Extension", "Dedication",
    "Planning Area Name"
]
df[cat_columns] = df[cat_columns].astype("category")



In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Technical ID          2556 non-null   object  
 1   Key                   2556 non-null   object  
 2   Object Number         2556 non-null   object  
 3   neighborhood          2556 non-null   category
 4   Locality              2556 non-null   category
 5   Type of Green Space   2556 non-null   category
 6   Green Space Name      2556 non-null   category
 7   Name Extension        2556 non-null   category
 8   Year Built            362 non-null    float64 
 9   Last Renovation Year  167 non-null    float64 
 10  Size sqm              2553 non-null   float64 
 11  Dedication            2556 non-null   category
 12  Planning Area Number  2555 non-null   float64 
 13  Planning Area Name    2556 non-null   category
dtypes: category(7), float64(4), object(3)
memory usage: 315.

### Handle Missing Values after change the datatype

In [15]:
df.isna().sum()

Technical ID               0
Key                        0
Object Number              0
neighborhood               0
Locality                   0
Type of Green Space        0
Green Space Name           0
Name Extension             0
Year Built              2194
Last Renovation Year    2389
Size sqm                   3
Dedication                 0
Planning Area Number       1
Planning Area Name         0
dtype: int64

* Fill missing values (imputation) For numeric columns:

In [16]:
df["Size sqm"] = df["Size sqm"].fillna(df["Size sqm"].median())
df["Year Built"] = df["Year Built"].fillna(df["Year Built"].median())
df["Planning Area Number"] = df["Planning Area Number"].fillna(df["Planning Area Number"].median())
df["Last Renovation Year"] = df["Last Renovation Year"].fillna(0)  # or use median if applicable


In [17]:
df.isna().sum()

Technical ID            0
Key                     0
Object Number           0
neighborhood            0
Locality                0
Type of Green Space     0
Green Space Name        0
Name Extension          0
Year Built              0
Last Renovation Year    0
Size sqm                0
Dedication              0
Planning Area Number    0
Planning Area Name      0
dtype: int64

### Create Address1 Column with Green Space Name

In [18]:
df['Address1'] = (
    df['Green Space Name'].astype(str) + ", Berlin, Germany" #change the datatype before apply the geocoding
)

df.head()

,Technical ID,Key,Object Number,neighborhood,Locality,Type of Green Space,Green Space Name,Name Extension,Year Built,Last Renovation Year,Size sqm,Dedication,Planning Area Number,Planning Area Name,Address1
0,00008100_001042bb,00008100:001042bb,00037,Reinickendorf,Frohnau,Grünanlage,"Im Fischgrund, ""Rosenanger""",Rosenanger,1995.0,0.0,16991.5,gewidmet,12400721.0,Frohnau Ost,"Im Fischgrund, ""Rosenanger"", Berlin, Germany"
1,00008100_00104621,00008100:00104621,01179,Reinickendorf,Lübars,Grünanlage,Klötzbecken bis Zabel-Krüger-Damm,einschl. Klötzbecken,1995.0,0.0,52224.6,gewidmet,12500929.0,Lübars,"Klötzbecken bis Zabel-Krüger-Damm, Berlin, Ger..."
2,00008100_001044bd,00008100:001044bd,01074,Reinickendorf,Hermsdorf,Grünanlage,"Heidenheimer Str. (ab Friedrichsthaler Weg), W...",-,1995.0,0.0,3012.0,gewidmet,12400722.0,Hermsdorf West,"Heidenheimer Str. (ab Friedrichsthaler Weg), W..."
3,00008100_00104620,00008100:00104620,01180,Reinickendorf,Lübars,Grünanlage,"Wittenauer Str., südl. AEG-Siedlung",-,1995.0,0.0,3374.2,gewidmet,12500929.0,Lübars,"Wittenauer Str., südl. AEG-Siedlung, Berlin, G..."
4,00008100_00104438,00008100:00104438,00476,Reinickendorf,Reinickendorf,Grünanlage,Kuhnpromenade u. Lindauer Allee 59/61,-,1995.0,0.0,3122.0,gewidmet,12100206.0,Humboldtstraße,"Kuhnpromenade u. Lindauer Allee 59/61, Berlin,..."


### Lets look for duplicates 

In [19]:
# Count duplicate Address values
duplicates = df['Address1'].duplicated().sum()
print(f"Found {duplicates} duplicate addresses.")

Found 19 duplicate addresses.


### Create a unique address DataFrame

In [20]:
unique_addresses = df[['Address1']].drop_duplicates().copy()
unique_addresses

,Address1
0,"Im Fischgrund, ""Rosenanger"", Berlin, Germany"
1,"Klötzbecken bis Zabel-Krüger-Damm, Berlin, Ger..."
2,"Heidenheimer Str. (ab Friedrichsthaler Weg), W..."
3,"Wittenauer Str., südl. AEG-Siedlung, Berlin, G..."
4,"Kuhnpromenade u. Lindauer Allee 59/61, Berlin,..."
...,...
2550,"Weddingplatz, Berlin, Germany"
2551,"Essener Park, Berlin, Germany"
2553,"Leopoldplatz an der Alten Nazarethkirche, Berl..."
2554,"Mollstr. 15-18, Berlin, Germany"


### Geocode only the sample_df (10 rows) of unique addresses using OpenStreetMap’s Nominatim API 

In [21]:
sample_df = unique_addresses.loc[0:10]
sample_df

,Address1
0,"Im Fischgrund, ""Rosenanger"", Berlin, Germany"
1,"Klötzbecken bis Zabel-Krüger-Damm, Berlin, Ger..."
2,"Heidenheimer Str. (ab Friedrichsthaler Weg), W..."
3,"Wittenauer Str., südl. AEG-Siedlung, Berlin, G..."
4,"Kuhnpromenade u. Lindauer Allee 59/61, Berlin,..."
5,"Avenue Charles de Gaulle 32-33, Berlin, Germany"
6,"Platz der US-Berlin-Brigaden WG, Berlin, Germany"
7,"Schünemannweg N, Berlin, Germany"
8,"Grabens. Hlgs., Lindengraben, Berlin, Germany"
9,"BAB, Überbauung Tunnel Tegel, Berlin, Germany"


In [22]:
geolocator = Nominatim(user_agent="berlin-geocoder")

def geocode_address(address):
    try:
        location = geolocator.geocode(address)
        sleep(1)
        if location:
            return pd.Series([location.latitude, location.longitude])
        else: 
            return pd.Series([None, None])
    except:
        return pd.Series([None, None])

In [23]:
# Geocode sample_df
sample_df[['Latitude', 'Longitude']] = sample_df['Address1'].apply(geocode_address)


/var/folders/p6/1v6w0vgj3d951b4r8sgj0lgw0000gn/T/ipykernel_11320/970622226.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df[['Latitude', 'Longitude']] = sample_df['Address1'].apply(geocode_address)
/var/folders/p6/1v6w0vgj3d951b4r8sgj0lgw0000gn/T/ipykernel_11320/970622226.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df[['Latitude', 'Longitude']] = sample_df['Address1'].apply(geocode_address)


In [24]:
sample_df

,Address1,Latitude,Longitude
0,"Im Fischgrund, ""Rosenanger"", Berlin, Germany",NaN,NaN
1,"Klötzbecken bis Zabel-Krüger-Damm, Berlin, Ger...",NaN,NaN
2,"Heidenheimer Str. (ab Friedrichsthaler Weg), W...",NaN,NaN
3,"Wittenauer Str., südl. AEG-Siedlung, Berlin, G...",NaN,NaN
4,"Kuhnpromenade u. Lindauer Allee 59/61, Berlin,...",NaN,NaN
5,"Avenue Charles de Gaulle 32-33, Berlin, Germany",52.601249,13.319022
6,"Platz der US-Berlin-Brigaden WG, Berlin, Germany",NaN,NaN
7,"Schünemannweg N, Berlin, Germany",52.444492,13.352584
8,"Grabens. Hlgs., Lindengraben, Berlin, Germany",NaN,NaN
9,"BAB, Überbauung Tunnel Tegel, Berlin, Germany",NaN,NaN


### Geocode the unique addresses of the entire dataset using OpenStreetMap’s Nominatim API 

In [25]:
geolocator = Nominatim(user_agent="berlin-geocoder")

def geocode_address(address):
    try:
        location = geolocator.geocode(address)
        sleep(1)
        if location:
            return pd.Series([location.latitude, location.longitude])
        else: 
            return pd.Series([None, None])
    except:
        return pd.Series([None, None])

In [26]:
# Geocode unique addresses
unique_addresses[['Latitude', 'Longitude']] = unique_addresses['Address1'].apply(geocode_address)

In [27]:
unique_addresses

,Address1,Latitude,Longitude
0,"Im Fischgrund, ""Rosenanger"", Berlin, Germany",NaN,NaN
1,"Klötzbecken bis Zabel-Krüger-Damm, Berlin, Ger...",NaN,NaN
2,"Heidenheimer Str. (ab Friedrichsthaler Weg), W...",NaN,NaN
3,"Wittenauer Str., südl. AEG-Siedlung, Berlin, G...",NaN,NaN
4,"Kuhnpromenade u. Lindauer Allee 59/61, Berlin,...",NaN,NaN
...,...,...,...
2550,"Weddingplatz, Berlin, Germany",52.540801,13.369760
2551,"Essener Park, Berlin, Germany",52.524730,13.340990
2553,"Leopoldplatz an der Alten Nazarethkirche, Berl...",NaN,NaN
2554,"Mollstr. 15-18, Berlin, Germany",52.523370,13.424753


In [28]:
df.head()

,Technical ID,Key,Object Number,neighborhood,Locality,Type of Green Space,Green Space Name,Name Extension,Year Built,Last Renovation Year,Size sqm,Dedication,Planning Area Number,Planning Area Name,Address1
0,00008100_001042bb,00008100:001042bb,00037,Reinickendorf,Frohnau,Grünanlage,"Im Fischgrund, ""Rosenanger""",Rosenanger,1995.0,0.0,16991.5,gewidmet,12400721.0,Frohnau Ost,"Im Fischgrund, ""Rosenanger"", Berlin, Germany"
1,00008100_00104621,00008100:00104621,01179,Reinickendorf,Lübars,Grünanlage,Klötzbecken bis Zabel-Krüger-Damm,einschl. Klötzbecken,1995.0,0.0,52224.6,gewidmet,12500929.0,Lübars,"Klötzbecken bis Zabel-Krüger-Damm, Berlin, Ger..."
2,00008100_001044bd,00008100:001044bd,01074,Reinickendorf,Hermsdorf,Grünanlage,"Heidenheimer Str. (ab Friedrichsthaler Weg), W...",-,1995.0,0.0,3012.0,gewidmet,12400722.0,Hermsdorf West,"Heidenheimer Str. (ab Friedrichsthaler Weg), W..."
3,00008100_00104620,00008100:00104620,01180,Reinickendorf,Lübars,Grünanlage,"Wittenauer Str., südl. AEG-Siedlung",-,1995.0,0.0,3374.2,gewidmet,12500929.0,Lübars,"Wittenauer Str., südl. AEG-Siedlung, Berlin, G..."
4,00008100_00104438,00008100:00104438,00476,Reinickendorf,Reinickendorf,Grünanlage,Kuhnpromenade u. Lindauer Allee 59/61,-,1995.0,0.0,3122.0,gewidmet,12100206.0,Humboldtstraße,"Kuhnpromenade u. Lindauer Allee 59/61, Berlin,..."


### For full workflow is recomendable to save the table into a csv for avoid the long wait (101 minutes) for run the geocoder with Nominatim API

1. After geocoding (takes 101 mins):

In [29]:
unique_addresses.to_csv("unique_addresses_geocoded.csv", index=False)

2. In future sessions (fast, ~2s):

In [30]:
unique_addresses = pd.read_csv("unique_addresses_geocoded.csv")

3. Then merge cleanly:

In [31]:
df = df.drop(columns=[col for col in df.columns if 'Latitude' in col or 'Longitude' in col], errors='ignore')
df = df.merge(unique_addresses[['Address1', 'Latitude', 'Longitude']], on='Address1', how='left')
df.head()

,Technical ID,Key,Object Number,neighborhood,Locality,Type of Green Space,Green Space Name,Name Extension,Year Built,Last Renovation Year,Size sqm,Dedication,Planning Area Number,Planning Area Name,Address1,Latitude,Longitude
0,00008100_001042bb,00008100:001042bb,00037,Reinickendorf,Frohnau,Grünanlage,"Im Fischgrund, ""Rosenanger""",Rosenanger,1995.0,0.0,16991.5,gewidmet,12400721.0,Frohnau Ost,"Im Fischgrund, ""Rosenanger"", Berlin, Germany",NaN,NaN
1,00008100_00104621,00008100:00104621,01179,Reinickendorf,Lübars,Grünanlage,Klötzbecken bis Zabel-Krüger-Damm,einschl. Klötzbecken,1995.0,0.0,52224.6,gewidmet,12500929.0,Lübars,"Klötzbecken bis Zabel-Krüger-Damm, Berlin, Ger...",NaN,NaN
2,00008100_001044bd,00008100:001044bd,01074,Reinickendorf,Hermsdorf,Grünanlage,"Heidenheimer Str. (ab Friedrichsthaler Weg), W...",-,1995.0,0.0,3012.0,gewidmet,12400722.0,Hermsdorf West,"Heidenheimer Str. (ab Friedrichsthaler Weg), W...",NaN,NaN
3,00008100_00104620,00008100:00104620,01180,Reinickendorf,Lübars,Grünanlage,"Wittenauer Str., südl. AEG-Siedlung",-,1995.0,0.0,3374.2,gewidmet,12500929.0,Lübars,"Wittenauer Str., südl. AEG-Siedlung, Berlin, G...",NaN,NaN
4,00008100_00104438,00008100:00104438,00476,Reinickendorf,Reinickendorf,Grünanlage,Kuhnpromenade u. Lindauer Allee 59/61,-,1995.0,0.0,3122.0,gewidmet,12100206.0,Humboldtstraße,"Kuhnpromenade u. Lindauer Allee 59/61, Berlin,...",NaN,NaN


### Change column names to snake_case

In [37]:
df.columns

Index(['Technical ID', 'Key', 'Object Number', 'neighborhood', 'Locality',
       'Type of Green Space', 'Green Space Name', 'Name Extension',
       'Year Built', 'Last Renovation Year', 'Size sqm', 'Dedication',
       'Planning Area Number', 'Planning Area Name', 'Address1', 'Latitude',
       'Longitude'],
      dtype='object')

In [38]:
def to_snake_case(col):
    col = col.strip()  # remove leading/trailing whitespace
    col = re.sub(r'[\s\-]+', '_', col)  # replace spaces and hyphens with underscores
    col = re.sub(r'([a-z])([A-Z])', r'\1_\2', col)  # add underscore between camelCase words
    col = col.lower()
    return col

In [39]:
df.columns = [to_snake_case(col) for col in df.columns]


In [40]:
df.columns

Index(['technical_id', 'key', 'object_number', 'neighborhood', 'locality',
       'type_of_green_space', 'green_space_name', 'name_extension',
       'year_built', 'last_renovation_year', 'size_sqm', 'dedication',
       'planning_area_number', 'planning_area_name', 'address1', 'latitude',
       'longitude'],
      dtype='object')